# 1

In [36]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')[0]
data.head()

,Postcode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [92]:
data = data.rename(columns={'Postcode': 'PostalCode','District':'Borough'})
data.head()

,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor


In [4]:
data = data.loc[(data['Borough'] != 'Not assigned')]
data.head()

,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor


In [6]:
data.loc[data['Neighborhood'] == 'Not assigned','Neighborhood'] = data.loc[data['Neighborhood'] == 'Not assigned','Borough']
data.head()

,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor


In [7]:
neighborhoods = data.groupby(['PostalCode','Borough'])['Neighborhood'].apply(', '.join).reset_index()
neighborhoods.shape

(103, 3)

In [8]:
neighborhoods.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


# 2

In [9]:
geodata_toronto = pd.read_csv('http://cocl.us/Geospatial_data')
geodata_toronto.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [ ]:
#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API la
#import geocoder # import geocoder

# initialize your variable to None
#lat_lng_coords = None

# loop until you get the coordinates
#while(lat_lng_coords is None):
  #g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
  #lat_lng_coords = g.latlng

#latitude = lat_lng_coords[0]
#longitude = lat_lng_coords[1]

In [10]:
neighborhoods = pd.merge(neighborhoods, geodata_toronto,left_on="PostalCode", right_on="Postal Code")

In [11]:
neighborhoods.shape

(103, 6)

In [12]:
neighborhoods.head()

,PostalCode,Borough,Neighborhood,Postal Code,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",M1B,43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",M1C,43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",M1E,43.763573,-79.188711
3,M1G,Scarborough,Woburn,M1G,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,M1H,43.773136,-79.239476


In [13]:
neighborhoods = neighborhoods.drop('Postal Code',axis = 1)

In [14]:
neighborhoods.head()


,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


# 3

In [16]:
# uncomment this line if you haven't completed the Foursquare API lab
! pip install folium
import folium # map rendering library

     |████████████████████████████████| 92kB 10.8MB/s eta 0:00:01


In [17]:
#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

address = 'Toronto'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [19]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=3,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [20]:
toronto = neighborhoods[neighborhoods['Borough'].str.contains('Toronto')].reset_index(drop=True)
toronto.head()
toronto.shape

(39, 5)

In [22]:
#latitude = 43.653908
#longitude = -79.384293
# create map of New York using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=13)

# add markers to map
for lat, lng, borough, neighborhood in zip(toronto['Latitude'], toronto['Longitude'], toronto['Borough'], toronto['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=3,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [23]:
CLIENT_ID = 'IFBX01UP1J24CUVKQW2TZ3ZYWWYBUECRGRGCFLFDV2SDOY2Z' # your Foursquare ID
CLIENT_SECRET = 'FOHS4F1PAADMGTAHEQCLGELE5GOZR5H4SQ2QBMWVYVEFWST5' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: IFBX01UP1J24CUVKQW2TZ3ZYWWYBUECRGRGCFLFDV2SDOY2Z
CLIENT_SECRET:FOHS4F1PAADMGTAHEQCLGELE5GOZR5H4SQ2QBMWVYVEFWST5


In [25]:
import json # library to handle JSON files

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

In [103]:
def getNearbyVenues(pcodes, names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for pcode, name, lat, lng in zip(pcodes, names, latitudes, longitudes):
        print(pcodes)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            pcode,
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['PostalCode',
                  'Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [104]:
toronto_venues = getNearbyVenues(pcodes = toronto['PostalCode'],
                                   names = toronto['Neighborhood'],
                                   latitudes = toronto['Latitude'],
                                   longitudes = toronto['Longitude']
                                  )


0     M4E
1     M4K
2     M4L
3     M4M
4     M4N
5     M4P
6     M4R
7     M4S
8     M4T
9     M4V
10    M4W
11    M4X
12    M4Y
13    M5A
14    M5B
15    M5C
16    M5E
17    M5G
18    M5H
19    M5J
20    M5K
21    M5L
22    M5N
23    M5P
24    M5R
25    M5S
26    M5T
27    M5V
28    M5W
29    M5X
30    M6G
31    M6H
32    M6J
33    M6K
34    M6P
35    M6R
36    M6S
37    M7Y
38    M9A
Name: PostalCode, dtype: object
0     M4E
1     M4K
2     M4L
3     M4M
4     M4N
5     M4P
6     M4R
7     M4S
8     M4T
9     M4V
10    M4W
11    M4X
12    M4Y
13    M5A
14    M5B
15    M5C
16    M5E
17    M5G
18    M5H
19    M5J
20    M5K
21    M5L
22    M5N
23    M5P
24    M5R
25    M5S
26    M5T
27    M5V
28    M5W
29    M5X
30    M6G
31    M6H
32    M6J
33    M6K
34    M6P
35    M6R
36    M6S
37    M7Y
38    M9A
Name: PostalCode, dtype: object
0     M4E
1     M4K
2     M4L
3     M4M
4     M4N
5     M4P
6     M4R
7     M4S
8     M4T
9     M4V
10    M4W
11    M4X
12    M4Y
13    M5A
14    M5B
15    

In [105]:
print(toronto_venues.shape)
toronto_venues.head()

(337, 8)


,PostalCode,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,M4E,The Beaches,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
1,M4E,The Beaches,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
2,M4E,The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
3,M4E,The Beaches,43.676357,-79.293031,Domino's Pizza,43.679058,-79.297382,Pizza Place
4,M4E,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood


In [106]:
toronto_venues.groupby('PostalCode').count()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
PostalCode,,,,,,,
M4E,5,5,5,5,5,5,5
M4K,10,10,10,10,10,10,10
M4L,10,10,10,10,10,10,10
M4M,10,10,10,10,10,10,10
M4N,3,3,3,3,3,3,3
M4P,8,8,8,8,8,8,8
M4R,10,10,10,10,10,10,10
M4S,10,10,10,10,10,10,10
M4T,2,2,2,2,2,2,2


In [107]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 119 uniques categories.


In [108]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['PostalCode'] = toronto_venues['PostalCode'] 

# move neighborhood column to the first column
toronto_onehot = toronto_onehot[ ['PostalCode'] + [ col for col in toronto_onehot.columns if col != 'PostalCode' ] ]

toronto_onehot.head() 
#toronto_onehot.shape

,PostalCode,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Terminal,American Restaurant,Arts & Crafts Store,Asian Restaurant,Auto Workshop,...,Sushi Restaurant,Swim School,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Trail,Vegetarian / Vegan Restaurant,Wine Bar,Yoga Studio
0,M4E,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
1,M4E,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,M4E,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,M4E,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,M4E,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [109]:
toronto_grouped = toronto_onehot.groupby('PostalCode').mean().reset_index()
toronto_grouped

,PostalCode,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Terminal,American Restaurant,Arts & Crafts Store,Asian Restaurant,Auto Workshop,...,Sushi Restaurant,Swim School,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Trail,Vegetarian / Vegan Restaurant,Wine Bar,Yoga Studio
0,M4E,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00,0.000000,0.0,0.0,0.0,0.0,0.20,0.0,0.0,0.0
1,M4K,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00,0.000000,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.1
2,M4L,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.10,0.000000,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0
3,M4M,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00,0.000000,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0
4,M4N,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00,0.333333,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0
5,M4P,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00,0.000000,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0
6,M4R,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00,0.000000,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.1
7,M4S,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.10,0.000000,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0
8,M4T,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00,0.000000,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0
9,M4V,0.0,0.0,0.0,0.0,0.0,0.1,0.0,0.0,0.0,...,0.10,0.000000,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0


In [111]:
num_top_venues = 5

for hood in toronto_grouped['PostalCode']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['PostalCode'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----M4E----
               venue  freq
0        Pizza Place   0.2
1              Trail   0.2
2       Neighborhood   0.2
3                Pub   0.2
4  Health Food Store   0.2


----M4K----
                     venue  freq
0         Greek Restaurant   0.3
1           Ice Cream Shop   0.2
2  Fruit & Vegetable Store   0.1
3           Cosmetics Shop   0.1
4                  Brewery   0.1


----M4L----
                venue  freq
0   Fish & Chips Shop   0.1
1        Burger Joint   0.1
2             Brewery   0.1
3           Pet Store   0.1
4  Italian Restaurant   0.1


----M4M----
          venue  freq
0   Coffee Shop   0.1
1        Bakery   0.1
2  Neighborhood   0.1
3   Cheese Shop   0.1
4          Café   0.1


----M4N----
               venue  freq
0               Park  0.33
1        Swim School  0.33
2           Bus Line  0.33
3            Airport  0.00
4  Korean Restaurant  0.00


----M4P----
               venue  freq
0                Gym  0.12
1              Hotel  0.12
2              

In [112]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [113]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['PostalCode']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['PostalCode'] = toronto_grouped['PostalCode']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,PostalCode,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,Neighborhood,Trail,Pizza Place,Health Food Store,Pub,Convenience Store,Dance Studio,Cuban Restaurant,Creperie,Cosmetics Shop
1,M4K,Greek Restaurant,Ice Cream Shop,Yoga Studio,Cosmetics Shop,Brewery,Italian Restaurant,Fruit & Vegetable Store,Fountain,Dog Run,College Gym
2,M4L,Park,Sushi Restaurant,Pet Store,Liquor Store,Burger Joint,Brewery,Italian Restaurant,Fish & Chips Shop,Ice Cream Shop,Gym
3,M4M,Coffee Shop,Café,Neighborhood,Bookstore,Ice Cream Shop,Bakery,Sandwich Place,Fish Market,Cheese Shop,Pet Store
4,M4N,Swim School,Bus Line,Park,Yoga Studio,Diner,Cocktail Bar,Coffee Shop,College Gym,Comic Shop,Concert Hall


In [114]:
from sklearn.cluster import KMeans

In [116]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('PostalCode', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_

array([3, 3, 3, 0, 2, 3, 3, 0, 4, 0, 2, 0, 3, 3, 3, 0, 3, 0, 3, 3, 0, 0,
       1, 2, 0, 0, 0, 3, 3, 0, 0, 0, 3, 0, 3, 0, 0, 3], dtype=int32)

In [117]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'ClusterLabels', kmeans.labels_)
neighborhoods_venues_sorted.head()



,ClusterLabels,PostalCode,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,3,M4E,Neighborhood,Trail,Pizza Place,Health Food Store,Pub,Convenience Store,Dance Studio,Cuban Restaurant,Creperie,Cosmetics Shop
1,3,M4K,Greek Restaurant,Ice Cream Shop,Yoga Studio,Cosmetics Shop,Brewery,Italian Restaurant,Fruit & Vegetable Store,Fountain,Dog Run,College Gym
2,3,M4L,Park,Sushi Restaurant,Pet Store,Liquor Store,Burger Joint,Brewery,Italian Restaurant,Fish & Chips Shop,Ice Cream Shop,Gym
3,0,M4M,Coffee Shop,Café,Neighborhood,Bookstore,Ice Cream Shop,Bakery,Sandwich Place,Fish Market,Cheese Shop,Pet Store
4,2,M4N,Swim School,Bus Line,Park,Yoga Studio,Diner,Cocktail Bar,Coffee Shop,College Gym,Comic Shop,Concert Hall


In [118]:
toronto_merged = toronto

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('PostalCode'), on='PostalCode')

toronto_merged.head() # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,ClusterLabels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,3.0,Neighborhood,Trail,Pizza Place,Health Food Store,Pub,Convenience Store,Dance Studio,Cuban Restaurant,Creperie,Cosmetics Shop
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188,3.0,Greek Restaurant,Ice Cream Shop,Yoga Studio,Cosmetics Shop,Brewery,Italian Restaurant,Fruit & Vegetable Store,Fountain,Dog Run,College Gym
2,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572,3.0,Park,Sushi Restaurant,Pet Store,Liquor Store,Burger Joint,Brewery,Italian Restaurant,Fish & Chips Shop,Ice Cream Shop,Gym
3,M4M,East Toronto,Studio District,43.659526,-79.340923,0.0,Coffee Shop,Café,Neighborhood,Bookstore,Ice Cream Shop,Bakery,Sandwich Place,Fish Market,Cheese Shop,Pet Store
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,2.0,Swim School,Bus Line,Park,Yoga Studio,Diner,Cocktail Bar,Coffee Shop,College Gym,Comic Shop,Concert Hall


In [120]:
import matplotlib.cm as cm
import matplotlib.colors as colors

In [122]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['ClusterLabels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

TypeError: list indices must be integers or slices, not float